# Pré-processamento de Imagens: Redimensionamento e Conversão para Escala de Cinza

Nessa etapa, vai ser realizado duas operações principais em uma imagem do dataset para as letras A, B e C:

1. **Redimensionamento das Imagens**: Todas as imagens serão redimensionadas para um tamanho fixo de `200x200 px` para ter uma consistência na análise posterior.
2. **Conversão para Escala de Cinza**: As imagens coloridas serão convertidas para **escala de cinza** para simplificar a análise, removendo as informações de cor.

## Passos:

1. **Leitura das Imagens**: As imagens de cada letra (A, B, C) foram lidas a partir da pasta correspondente.
2. **Redimensionamento**: Cada imagem foi redimensionada para `200x200 px` usando o OpenCV.
3. **Conversão para Escala de Cinza**: As imagens foram convertidas para escala de cinza para reduzir a complexidade e facilitar o processamento posterior.

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from pathlib import Path

base_path = Path("dataset")
letters = ['A', 'B', 'C']
output_path = Path("images_result")

def resize(path_img):
    img = cv2.imread(str(path_img))
    img_resize = cv2.resize(img, (200, 200))
    return img_resize

def to_gray_scale(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img_gray

def process_img(path_img):
    img_resized = resize(path_img)
    img_gray_resized = to_gray_scale(img_resized)
    return img_gray_resized

for letter in letters:
    dir = base_path / letter

    img = os.listdir(dir)[0]
    img_path = dir / img

    processed_img = process_img(img_path)

    plt.imshow(processed_img, cmap='gray')
    plt.title(f'Letra {letter}')
    plt.axis('off')
    plt.show()

    out_file = output_path / 'resized_gray' / f'{letter}.png'
    cv2.imwrite(str(out_file), processed_img)
